In [1]:
using BenchmarkTools
using Combinatorics
using HypothesisTests
using Memoization
using StatsBase

$\{1,2,\ldots,N\}$ から重複無しに $m$ 個の組み合わせ $1\le i_1<\cdots<i_m\le N$ を取るとき, その和が $r$ になる場合の数を $f(N, m, r)$ と書くと, 以下が成立することがわかる:

$$
\begin{aligned}
&
f(N, m, r) = 0\quad\text{unless $0\le m \le N$ and $0\le r - m(m+1)/2 \le m(N-m)$},
\\ &
f(N, 0, 0) = 1,
\\ &
f(N, 1, r) = \text{if $1\le r\le N$ then $1$ else $0$},
\\ &
f(N, m, r) = f(N-1, m, r) + f(N-1, m-1, r-N).
\end{aligned}
$$

ただし, この方法で計算するにはメモ化がほぼ必須である.

以下のセルのコードは上の方法による $f(N, m, r)$ の計算法をそのままコードに翻訳したものである.

In [2]:
function f_naive(N, m, r)
    0 ≤ m ≤ N || return zero(N)
    0 ≤ r - m*(m+1)÷2 ≤ m*(N-m) || return zero(N)
    m == 0 && return one(N)
    m == 1 && return 1 ≤ r ≤ N ? one(N) : zero(N)
    f_naive(N-1, m, r) + f_naive(N-1, m-1, r-N)
end

@memoize function f_memoize(N, m, r)
    0 ≤ m ≤ N || return zero(N)
    0 ≤ r - m*(m+1)÷2 ≤ m*(N-m) || return zero(N)
    m == 0 && return one(N)
    m == 1 && return 1 ≤ r ≤ N ? one(N) : zero(N)
    f_memoize(N-1, m, r) + f_memoize(N-1, m-1, r-N)
end

f_memoize (generic function with 1 method)

メモ化していない `f_naive` はひどく遅い.

In [3]:
for m in 10:18
    N = 2m
    r = m*(m+1)÷2 + m*(N-m)÷2
    print("m = $m, N = $N, r = $r: ")
    @time f_naive(N, m, r)
end

m = 10, N = 20, r = 105:   0.000163 seconds
m = 11, N = 22, r = 126:   0.000444 seconds
m = 12, N = 24, r = 150:   0.001466 seconds
m = 13, N = 26, r = 175:   0.005098 seconds
m = 14, N = 28, r = 203:   0.017502 seconds
m = 15, N = 30, r = 232:   0.061890 seconds
m = 16, N = 32, r = 264:   0.237123 seconds
m = 17, N = 34, r = 297:   0.774752 seconds
m = 18, N = 36, r = 333:   2.698451 seconds


メモ化された `f` はこれよりかなり速くなっている.

In [4]:
for m in 10:18
    N = 2m
    r = m*(m+1)÷2 + m*(N-m)÷2
    print("m = $m, N = $N, r = $r: ")
    @time f_memoize(N, m, r)
end

m = 10, N = 20, r = 105:   0.013240 seconds (4.74 k allocations: 298.744 KiB, 98.55% compilation time)
m = 11, N = 22, r = 126:   0.000079 seconds (896 allocations: 27.094 KiB)
m = 12, N = 24, r = 150:   0.000138 seconds (1.48 k allocations: 44.312 KiB)
m = 13, N = 26, r = 175:   0.000113 seconds (1.62 k allocations: 47.719 KiB)
m = 14, N = 28, r = 203:   0.000223 seconds (2.45 k allocations: 71.375 KiB)
m = 15, N = 30, r = 232:   0.000428 seconds (2.78 k allocations: 334.672 KiB)
m = 16, N = 32, r = 264:   0.000264 seconds (3.83 k allocations: 109.281 KiB)
m = 17, N = 34, r = 297:   0.000260 seconds (4.24 k allocations: 117.812 KiB)
m = 18, N = 36, r = 333:   0.001569 seconds (5.72 k allocations: 160.094 KiB)


念のために正しく計算されていることを確認しておこう.

In [5]:
function F_memoize(N, m)
    rmin = m*(m+1)÷2
    rmax = rmin + m*(N-m)
    rs = rmin:rmax
    fs = [f_memoize(N, m, r) for r in rs]
    rs, fs
end

function F_comb(N, m)
    rank_sums = [sum(comb) for comb in combinations(1:N, m)]
    c = countmap(rank_sums)
    rmin, rmax = extrema(keys(c))
    rs = rmin:rmax
    fs = [c[k] for k in rs]
    rs, fs
end

F_comb (generic function with 1 method)

In [6]:
N = 10
F1 = F_memoize
F2 = F_comb
for m in 0:N
    @show N, m, F1(N, m)
    @show N, m, F2(N, m)
    @show F1(N, m) == F2(N, m)
    println()
end

(N, m, F1(N, m)) = (10, 0, (0:0, [1]))
(N, m, F2(N, m)) = (10, 0, (0:0, [1]))
F1(N, m) == F2(N, m) = true

(N, m, F1(N, m)) = (10, 1, (1:10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
(N, m, F2(N, m)) = (10, 1, (1:10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
F1(N, m) == F2(N, m) = true

(N, m, F1(N, m)) = (10, 2, (3:19, [1, 1, 2, 2, 3, 3, 4, 4, 5, 4, 4, 3, 3, 2, 2, 1, 1]))
(N, m, F2(N, m)) = (10, 2, (3:19, [1, 1, 2, 2, 3, 3, 4, 4, 5, 4, 4, 3, 3, 2, 2, 1, 1]))
F1(N, m) == F2(N, m) = true

(N, m, F1(N, m)) = (10, 3, (6:27, [1, 1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 10, 10, 9, 8, 7, 5, 4, 3, 2, 1, 1]))
(N, m, F2(N, m)) = (10, 3, (6:27, [1, 1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 10, 10, 9, 8, 7, 5, 4, 3, 2, 1, 1]))
F1(N, m) == F2(N, m) = true

(N, m, F1(N, m)) = (10, 4, (10:34, [1, 1, 2, 3, 5, 6, 9, 10, 13, 14, 16, 16, 18, 16, 16, 14, 13, 10, 9, 6, 5, 3, 2, 1, 1]))
(N, m, F2(N, m)) = (10, 4, (10:34, [1, 1, 2, 3, 5, 6, 9, 10, 13, 14, 16, 16, 18, 16, 16, 14, 13, 10, 9, 6, 5, 3, 2, 1, 1]))
F1(N, m) == F2(N, m) = true

(N, m

In [7]:
[F_memoize(N, m) == F_comb(N, m) for N in 10:20, m in 0:10]

11×11 Matrix{Bool}:
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1

In [8]:
for N in 50:10:100
    m = N÷2
    print("N = $N, m = $m: ")
    @time F_memoize(N, m)
end

N = 50, m = 25:   0.048714 seconds (552.18 k allocations: 19.467 MiB)
N = 60, m = 30:   0.127367 seconds (642.06 k allocations: 16.590 MiB, 19.63% gc time)
N = 70, m = 35:   0.248481 seconds (1.06 M allocations: 35.240 MiB)
N = 80, m = 40:   0.993680 seconds (1.62 M allocations: 57.670 MiB, 13.68% gc time)
N = 90, m = 45:   2.860018 seconds (2.36 M allocations: 92.451 MiB, 4.39% gc time)
N = 100, m = 50:   0.791395 seconds (3.29 M allocations: 84.157 MiB, 33.52% gc time)


$$
\begin{aligned}
&
f(N, m, r) = 0\quad\text{unless $0\le m \le N$ and $0\le r - m(m+1)/2 \le m(N-m)$},
\\ &
f(N, 0, 0) = 1,
\\ &
f(N, 1, r) = \text{if $1\le r\le N$ then $1$ else $0$},
\\ &
f(N, m, r) = f(N-1, m, r) + f(N-1, m-1, r-N).
\end{aligned}
$$

より,

$$
N = m + n, \quad
r = m(m+1)/2 + u, \quad
g(n, m, u) = f(m+n, m, m(m+1)/2 + u)
$$

とおくと,

$$
\begin{aligned}
&
g(n, m, u) = 0\quad\text{unless $0 \le m$, $0\le n$, and $0\le u \le mn$},
\\ &
g(n, 0, 0) = 1,
\\ &
g(n, 1, u) = \text{if $0\le u \le m+n-1$ then $1$ else $0$},
\\ &
g(n, m, u) = g(n-1, m, u) + g(n, m-1, u-n).
\end{aligned}
$$

特に,

$$
g(0, 1, u) = \text{if $u=0$ then $1$ else $0$}.
$$

$g(n, m, u)$ の値は以下を帰納的に計算すれば得られる:

$$
g(j, i, u-(n-j)n), \quad 0\le j\le n, \quad 0\le i \le m.
$$

In [9]:
function G(n, m,
        prev = Matrix{Int}(undef, m*n+1, m+1),
        next = similar(prev),
    )
    # g(0, i, u) = δ_{i0} δ_{u0}
    @. next = 0
    next[1+0, 1+0] = 1
    for k in 1:m+n
        @. prev = next
        for i in max(0, k-n):min(m, k)
            j = k - i
            # g(j, i, u) = g(j-1, i, u) + g(j, i-1, u-j)
            @views @. next[1:1+i*(j-1), 1+i] = prev[1:1+i*(j-1), 1+i]
            if i ≥ 1
                @views @. next[1+j:1+i*j, 1+i] += prev[1:1+(i-1)*j, 1+i-1]
            end
            # g(0, i, u) = δ_{i0} δ_{u0}
            if j == 0
                next[1+0, 1+i] = 1
            end
        end
    end
    @view next[:, 1+m]
end

function F_recursive(N, m,
        prev = Matrix{Int}(undef, m*(N-m)+1, m+1),
        next = similar(prev),
    )
    n = N - m
    rmin = m*(m+1)÷2
    rmax = rmin + m*n
    rs = rmin:rmax
    fs = G(n, m, prev, next)
    rs, fs
end

F_recursive (generic function with 3 methods)

In [10]:
N = 10
F1 = F_memoize
F2 = F_recursive
for m in 0:N
    @show N, m, F1(N, m)
    @show N, m, F2(N, m)
    @show F1(N, m) == F2(N, m)
    println()
end

(N, m, F1(N, m)) = (10, 0, (0:0, [1]))
(N, m, F2(N, m)) = (10, 0, (0:0, [1]))
F1(N, m) == F2(N, m) = true

(N, m, F1(N, m)) = (10, 1, (1:10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
(N, m, F2(N, m)) = (10, 1, (1:10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
F1(N, m) == F2(N, m) = true

(N, m, F1(N, m)) = (10, 2, (3:19, [1, 1, 2, 2, 3, 3, 4, 4, 5, 4, 4, 3, 3, 2, 2, 1, 1]))
(N, m, F2(N, m)) = (10, 2, (3:19, [1, 1, 2, 2, 3, 3, 4, 4, 5, 4, 4, 3, 3, 2, 2, 1, 1]))
F1(N, m) == F2(N, m) = true

(N, m, F1(N, m)) = (10, 3, (6:27, [1, 1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 10, 10, 9, 8, 7, 5, 4, 3, 2, 1, 1]))
(N, m, F2(N, m)) = (10, 3, (6:27, [1, 1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 10, 10, 9, 8, 7, 5, 4, 3, 2, 1, 1]))
F1(N, m) == F2(N, m) = true

(N, m, F1(N, m)) = (10, 4, (10:34, [1, 1, 2, 3, 5, 6, 9, 10, 13, 14, 16, 16, 18, 16, 16, 14, 13, 10, 9, 6, 5, 3, 2, 1, 1]))
(N, m, F2(N, m)) = (10, 4, (10:34, [1, 1, 2, 3, 5, 6, 9, 10, 13, 14, 16, 16, 18, 16, 16, 14, 13, 10, 9, 6, 5, 3, 2, 1, 1]))
F1(N, m) == F2(N, m) = true

(N, m

In [11]:
[F_recursive(m+n, m) == F_memoize(m+n, m) for n in 0:20, m in 0:20]

21×21 Matrix{Bool}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1

In [12]:
for N in 50:10:200
    m = N÷2
    n = N - m
    print("N = $N, m = $m: ")
    prev = Matrix{Int}(undef, m*n+1, m+1)
    next = similar(prev)
    @time G(n, m, prev, next)
end

N = 50, m = 25:   0.000383 seconds
N = 60, m = 30:   0.000828 seconds
N = 70, m = 35:   0.002914 seconds
N = 80, m = 40:   0.002895 seconds
N = 90, m = 45:   0.004289 seconds
N = 100, m = 50:   0.006858 seconds
N = 110, m = 55:   0.009430 seconds
N = 120, m = 60:   0.013234 seconds
N = 130, m = 65:   0.017472 seconds
N = 140, m = 70:   0.023017 seconds
N = 150, m = 75:   0.038405 seconds
N = 160, m = 80:   0.041294 seconds
N = 170, m = 85:   0.051760 seconds
N = 180, m = 90:   0.076699 seconds
N = 190, m = 95:   0.107735 seconds
N = 200, m = 100:   0.144008 seconds


In [13]:
n, m = 100, 100
prev = Matrix{Int}(undef, m*n+1, m+1)
next = similar(prev)
@btime F_recursive($(m+n), $m, $prev, $next);

  93.647 ms (0 allocations: 0 bytes)


In [14]:
n, m = 100, 100
x, y = rand(m), rand(n)
@time ExactMannWhitneyUTest(x, y)
@time ExactMannWhitneyUTest(x, y)
@time ExactMannWhitneyUTest(x, y)

  0.147903 seconds (446.22 k allocations: 23.901 MiB, 99.85% compilation time)
  0.000020 seconds (7 allocations: 7.125 KiB)
  0.000019 seconds (7 allocations: 7.125 KiB)


Exact Mann-Whitney U test
-------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          0.0283697

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.7082

Details:
    number of observations in each group: [100, 100]
    Mann-Whitney-U statistic:             5154.0
    rank sums:                            [10204.0, 9896.0]
    adjustment for ties:                  0.0
